In [ ]:
# ============================
# STEP 1: Setup Environment
# ============================
!pip install ultralytics roboflow -q

from roboflow import Roboflow
from ultralytics import YOLO
import torch

print("Torch version:", torch.__version__)

In [ ]:
# ============================
# STEP 2: Connect Roboflow Dataset
# ============================
# Get dataset from Roboflow (public dataset example)
# Go to https://roboflow.com/datasets, pick one, and click "Download -> YOLOv8 -> Copy Code"

rf = Roboflow(api_key="YOUR_API_KEY")  # replace with your Roboflow API key
project = rf.workspace("roboflow-public-datasets").project("blood-cells")  # example public dataset
dataset = project.version(2).download("yolov8")

In [ ]:
# ============================
# STEP 3: Train YOLOv8 Model
# ============================

# Load a pretrained YOLOv8 model
# Options:
# yolov8n (nano, fastest)
# yolov8s (small)
# yolov8m (medium)
# yolov8l (large)
# yolov8x (x-large, most accurate)
model = YOLO("yolov8n.pt")

# Train the model on your dataset
model.train(
    data=dataset.location + "/data.yaml",  # Path to dataset config file; defines train/val image paths and class names
    epochs=50,                             # Number of full training passes through the dataset
    imgsz=640,                             # Image size (in pixels) YOLO resizes all images to before training
    batch=16,                              # Number of images processed per iteration (adjust based on GPU memory)
    name="yolov8",                         # Name of this training run; saved under runs/detect/yolov8/
    patience=10                            # Early stopping: stop if no validation improvement after 10 epochs
)


In [ ]:
# ============================
# STEP 4: Evaluate & Export
# ============================
# Evaluate model on validation set
model.val()

# Export the trained YOLOv8 model to different formats for deployment
# Common export formats:
# - "pt"          → PyTorch format (default, used for further YOLOv8 training or inference)
# - "onnx"        → Open Neural Network Exchange (for interoperability, e.g. OpenCV, TensorRT, etc.)
# - "torchscript" → For embedding models into C++/mobile environments
# - "engine"      → TensorRT engine (optimized GPU inference)
# - "coreml"      → Apple CoreML format (for iOS/macOS deployment)
# - "tflite"      → TensorFlow Lite (for Android, embedded devices)
# - "pb"          → TensorFlow SavedModel format
# - "xml"         → OpenVINO IR format (for Intel hardware)
# - "ncnn"        → NCNN format (for lightweight inference on edge devices)
# - "edgetpu"     → TensorFlow EdgeTPU format (for Coral EdgeTPU accelerators)
# - "tfjs"        → TensorFlow.js (for running in web browsers)

model.export(format="onnx")  # Example: export model to ONNX format

In [ ]:
# ============================
# STEP 5: Test on Sample Image
# ============================

# Run inference (prediction) using the trained model
results = model.predict(
    source=dataset.location + "/test/images",  # Path to the folder or file to run detection on
                                                # Can also be a single image path, video file, webcam index (0), or URL
    conf=0.5,                                   # Confidence threshold: ignore detections below this probability
    save=True                                   # Save output images/videos with bounding boxes and labels to 'runs/predict/'
)

# Print detection summary (class names, confidences, bounding boxes, etc.)
print(results)
# To view saved prediction images:
# Files will be stored in runs/predict/ — open that folder in Colab file explorer to preview results.
